In [2]:
import math
from pprint import pprint
from math import log
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter, xor
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import matrix
import mathematics
from importlib import reload

for m in (arrays, trees, lists, graphs, search, strings, matrix, mathematics):
    reload(m)

import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
import matrix as mt
import mathematics as m

print()

In [5]:
def candy_tree(r):
    "Equalize values in the tree to one. Count minimum number of moves."
    moves = 0
    def dfs(n):
        nonlocal moves
        c = n.data
        lc = dfs(n.left) if n.left else 1
        c += lc - 1
        moves += abs(lc - 1)
        rc = dfs(n.right) if n.right else 1
        c += rc - 1
        moves += abs(rc - 1)

        return c

    assert dfs(r) == 1
    return moves


In [6]:
r = tr.make_bfo("3 0 0")
r.display()
candy_tree(r)

 3 
/ \
0 0


2

In [7]:
r = tr.make_bfo("0 3 0")
r.display()
candy_tree(r)

 0 
/ \
3 0


3